# Moving from Manhattan to Cambridge


A young professional working at Columbia University has just accepted a new position at Harvard University. She loves her current community in the Manhattan neighborhood of Morningside Heights, but is now looking to move from New York City, NY to Cambridge, MA.  Before she moves, she wants to learn more about where she will be living and find a location to live that is (1) similar to her current Manhattan neighborhood of Morningside Heights and (2) has as many Coffee Shops as possible.

To get started, import the necessary libraries.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


# 1. Download and Transform the Data

Data for New York Neighborhoods is available online at https://geo.nyu.edu/catalog/nyu_2451_34572.  Data for Cambridge Neighborhoods is available online at https://geo.nyu.edu/catalog/harvard-cambridge14cddneighborhoods.

Download and open the datasets, extract the features, and load them into a dataframe.

a. New York Neighborhoods

In [2]:
#get the NY data
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
#load the NY data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
#extract relevant json
neighborhoods_data = newyork_data['features']

In [5]:
#transform the data into a pandas dataframe
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
#add borough, neighborhood, latitude, and longitude to the dataframe
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

b. Cambridge Neighborhoods

In [7]:
#get the cambridge data
!wget -q -O 'harvard-cambridge14cddneighborhoods-geojson.json' https://geo.nyu.edu/download/file/harvard-cambridge14cddneighborhoods-geojson.json
print('Data downloaded!')

Data downloaded!


In [8]:
#load the cambridge data
with open('harvard-cambridge14cddneighborhoods-geojson.json') as json_data:
    mass_data = json.load(json_data)

In [9]:
#extract relevant json
ma_neighborhoods_data = mass_data['features']

In [10]:
#add cambridge borough, neighborhood, latitude, and longitude to the existing dataframe
for data in ma_neighborhoods_data:
    borough = neighborhood_name = "Cambridge" 
    neighborhood_name = data['properties']['NAME']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[0][0][0][1]
    neighborhood_lon = neighborhood_latlon[0][0][0][0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [11]:
# view the full neighborhoods dataframe
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [17]:
#select manhattan and cambridge neighborhoods
#create a dataframe with both neighborhoods
all_data = neighborhoods[neighborhoods['Borough'].isin (['Manhattan', 'Cambridge'])].reset_index(drop=True)
all_data

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


In [18]:
#create a dataframe with only manhattan neighborhoods
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688
5,Manhattan,Manhattanville,40.816934,-73.957385
6,Manhattan,Central Harlem,40.815976,-73.943211
7,Manhattan,East Harlem,40.792249,-73.944182
8,Manhattan,Upper East Side,40.775639,-73.960508
9,Manhattan,Yorkville,40.775930,-73.947118


In [19]:
#create a dataframe with only cambridge neighborhoods
cambridge_data = neighborhoods[neighborhoods['Borough'] == 'Cambridge'].reset_index(drop=True)
cambridge_data

,Borough,Neighborhood,Latitude,Longitude
0,Cambridge,North Cambridge,42.388314,-71.119452
1,Cambridge,Neighborhood Nine,42.390271,-71.130443
2,Cambridge,Area 2/MIT,42.364903,-71.089729
3,Cambridge,Cambridgeport,42.365405,-71.103708
4,Cambridge,Riverside,42.365574,-71.103991
5,Cambridge,Mid-Cambridge,42.378088,-71.107275
6,Cambridge,Wellington-Harrington,42.371835,-71.098528
7,Cambridge,Area Four,42.371835,-71.098528
8,Cambridge,East Cambridge,42.369655,-71.088493
9,Cambridge,Agassiz,42.388314,-71.119452


# 2. Visualize Manhattan and Cambridge Neighborhoods

In [20]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
mlatitude = location.latitude
mlongitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(mlatitude, mlongitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [21]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[mlatitude, mlongitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [22]:
address = 'Cambridge, MA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
clatitude = location.latitude
clongitude = location.longitude
print('The geograpical coordinate of Cambridge are {}, {}.'.format(clatitude, clongitude))

The geograpical coordinate of Cambridge are 42.3750997, -71.1056157.


In [23]:
# create map of Cambridge using latitude and longitude values
map_cambridge = folium.Map(location=[clatitude, clongitude], zoom_start=13)

# add markers to map
for lat, lng, label in zip(cambridge_data['Latitude'], cambridge_data['Longitude'], cambridge_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cambridge)  
    
map_cambridge

# 3. Explore the neighborhoods

In [24]:
#define foursquare credentials and versions
#hidden cell below for security

In [25]:
# @hidden_cell
CLIENT_ID = 'EOMT2YJ5SO5UVRCLL1KWUEQ0XKTH5RQSVDZYTJ54UVUF24MJ' # your Foursquare ID
CLIENT_SECRET = '44UODM5X4SPNUU41PVRDIREIOE1V3YT0MMDNETTYDG01A40W' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [26]:
#create a function to explore Manhattan and Cambridge neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
#run the function on each neighborhood and create a new dataframe
LIMIT=100 #set limit of venues returned
all_venues = getNearbyVenues(names=all_data['Neighborhood'],
                                   latitudes=all_data['Latitude'],
                                   longitudes=all_data['Longitude']
                                  )
all_venues.head()

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
North Cambridge
Neighborhood Nine
Area 2/MIT
Cambridgeport
Riverside
Mid-Cambridge
Wellington-Harrington
Area Four
East Cambridge
Agassiz
Cambridge Highlands
Strawberry Hill
West Cambridge


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


In [28]:
#see how many venues were returned for each neighborhood
all_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agassiz,34,34,34,34,34,34
Area 2/MIT,39,39,39,39,39,39
Area Four,52,52,52,52,52,52
Battery Park City,100,100,100,100,100,100
Cambridge Highlands,9,9,9,9,9,9
Cambridgeport,49,49,49,49,49,49
Carnegie Hill,100,100,100,100,100,100
Central Harlem,43,43,43,43,43,43
Chelsea,100,100,100,100,100,100


# 4. Analyze the Neighborhoods

In [29]:
# one hot encoding
neighborhood_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = all_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Engineering Building,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandw

In [30]:
#check the size of the onehot dataframe
neighborhood_onehot.shape

(3739, 347)

In [31]:
#Group rows by neighborhood and take the mean of the frequency of occurrence of each category
all_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
all_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Engineering Building,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandw

In [32]:
#confirm the new size
all_grouped.shape

(53, 347)

In [33]:
#write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
#create a dataframe with the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = all_grouped['Neighborhood']

for ind in np.arange(all_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(all_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agassiz,Japanese Restaurant,Spa,Bar,Coffee Shop,Yoga Studio,Thai Restaurant,Liquor Store,Gym,Market,Gift Shop
1,Area 2/MIT,Café,Bar,Coffee Shop,Italian Restaurant,Hotel,Pizza Place,Park,Bookstore,Sandwich Place,Seafood Restaurant
2,Area Four,Coffee Shop,Park,Bar,New American Restaurant,Food & Drink Shop,Mediterranean Restaurant,Gastropub,Brazilian Restaurant,Sandwich Place,Chinese Restaurant
3,Battery Park City,Park,Coffee Shop,Hotel,Gym,Clothing Store,Italian Restaurant,Wine Shop,Pizza Place,BBQ Joint,Women's Store
4,Cambridge Highlands,Gourmet Shop,Deli / Bodega,Trail,Donut Shop,Art Gallery,Pet Store,Arts & Crafts Store,Hardware Store,Athletics & Sports,Yoga Studio


# 5. Cluster the Neighborhoods

In [35]:
#run k-means to cluster the neighborhood into 5 clusters
# set number of clusters
kclusters = 5

all_grouped_clustering = all_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(all_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 0, 0, 0, 0, 3, 0, 0, 0, 0], dtype=int32)

In [36]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

all_merged = all_data

# merge all_grouped with all_data to add latitude/longitude for each neighborhood
all_merged = all_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

all_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Discount Store,Coffee Shop,Kids Store,Big Box Store,Seafood Restaurant,Shoe Store,Tennis Stadium,Gym,Sandwich Place,Bank
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Bar,Ice Cream Shop,Bubble Tea Shop,Noodle House,Dumpling Restaurant,Bakery
2,Manhattan,Washington Heights,40.851903,-73.936900,4,Café,Bakery,Mobile Phone Shop,Grocery Store,Supermarket,New American Restaurant,Latin American Restaurant,Shoe Store,Tapas Restaurant,Park
3,Manhattan,Inwood,40.867684,-73.921210,4,Mexican Restaurant,Café,Lounge,Pizza Place,Park,Bakery,Frozen Yogurt Shop,Wine Bar,Chinese Restaurant,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,4,Mexican Restaurant,Coffee Shop,Pizza Place,Café,Liquor Store,Indian Restaurant,Sushi Restaurant,Deli / Bodega,Sandwich Place,Yoga Studio
5,Manhattan,Manhattanville,40.816934,-73.957385,0,Coffee Shop,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,Seafood Restaurant,Park,Deli / Bodega,Sushi Restaurant,Supermarket,Burger Joint
6,Manhattan,Central Harlem,40.815976,-73.943211,0,African Restaurant,French Restaurant,Chinese Restaurant,Gym / Fitness Center,Cosmetics Shop,Public Art,Seafood Restaurant,American Restaurant,Music Venue,Café
7,Manhattan,East Harlem,40.792249,-73.944182,4,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Donut Shop,Restaurant,Beer Bar,Clothing Store,Street Art
8,Manhattan,Upper East Side,40.775639,-73.960508,0,Italian Restaurant,Exhibit,Coffee Shop,Art Gallery,Bakery,Gym / Fitness Center,Juice Bar,Cocktail Bar,French Restaurant,Hotel
9,Manhattan,Yorkville,40.775930,-73.947118,0,Italian Restaurant,Bar,Gym,Coffee Shop,Pizza Place,Sushi Restaurant,Wine Shop,Diner,Ice Cream Shop,Japanese Restaurant


# 6. Isolate the appopriate cluster

In [37]:
#find the cluster label for the Morningside Heights neighborhood
morningside = all_merged.loc[all_merged['Neighborhood'] == 'Morningside Heights']
morningside_cluster=morningside['Cluster Labels'].values[0]
morningside_cluster

0

In [38]:
#examine the full cluster that contains the Morningside Heights neighborhood
chosen_cluster = all_merged[all_merged['Cluster Labels'] == morningside_cluster].reset_index(drop=True)
chosen_cluster

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Discount Store,Coffee Shop,Kids Store,Big Box Store,Seafood Restaurant,Shoe Store,Tennis Stadium,Gym,Sandwich Place,Bank
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,American Restaurant,Cocktail Bar,Vietnamese Restaurant,Bar,Ice Cream Shop,Bubble Tea Shop,Noodle House,Dumpling Restaurant,Bakery
2,Manhattan,Manhattanville,40.816934,-73.957385,0,Coffee Shop,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,Seafood Restaurant,Park,Deli / Bodega,Sushi Restaurant,Supermarket,Burger Joint
3,Manhattan,Central Harlem,40.815976,-73.943211,0,African Restaurant,French Restaurant,Chinese Restaurant,Gym / Fitness Center,Cosmetics Shop,Public Art,Seafood Restaurant,American Restaurant,Music Venue,Café
4,Manhattan,Upper East Side,40.775639,-73.960508,0,Italian Restaurant,Exhibit,Coffee Shop,Art Gallery,Bakery,Gym / Fitness Center,Juice Bar,Cocktail Bar,French Restaurant,Hotel
5,Manhattan,Yorkville,40.775930,-73.947118,0,Italian Restaurant,Bar,Gym,Coffee Shop,Pizza Place,Sushi Restaurant,Wine Shop,Diner,Ice Cream Shop,Japanese Restaurant
6,Manhattan,Lenox Hill,40.768113,-73.958860,0,Italian Restaurant,Coffee Shop,Sushi Restaurant,Gym / Fitness Center,Pizza Place,Burger Joint,Sporting Goods Shop,Gym,Thai Restaurant,Bakery
7,Manhattan,Roosevelt Island,40.762160,-73.949168,0,Sandwich Place,Coffee Shop,Deli / Bodega,Gym,Café,Park,Dog Run,School,Greek Restaurant,Supermarket
8,Manhattan,Upper West Side,40.787658,-73.977059,0,Italian Restaurant,Bar,Wine Bar,Mediterranean Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Burger Joint,Indian Restaurant,Bakery,Sushi Restaurant
9,Manhattan,Lincoln Square,40.773529,-73.985338,0,Gym / Fitness Center,Theater,Café,Concert Hall,Plaza,Italian Restaurant,Opera House,French Restaurant,Performing Arts Venue,Park


In [39]:
#extract the Cambridge neighborhoods
chosen_cluster_cambridge = chosen_cluster[chosen_cluster['Borough'] == 'Cambridge'].reset_index(drop=True)
chosen_cluster_cambridge

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Cambridge,Area 2/MIT,42.364903,-71.089729,0,Café,Bar,Coffee Shop,Italian Restaurant,Hotel,Pizza Place,Park,Bookstore,Sandwich Place,Seafood Restaurant
1,Cambridge,Mid-Cambridge,42.378088,-71.107275,0,Grocery Store,Coffee Shop,Climbing Gym,New American Restaurant,Bus Stop,Gourmet Shop,Thai Restaurant,Bed & Breakfast,Tapas Restaurant,Men's Store
2,Cambridge,Wellington-Harrington,42.371835,-71.098528,0,Coffee Shop,Park,Bar,New American Restaurant,Food & Drink Shop,Mediterranean Restaurant,Gastropub,Brazilian Restaurant,Sandwich Place,Chinese Restaurant
3,Cambridge,Area Four,42.371835,-71.098528,0,Coffee Shop,Park,Bar,New American Restaurant,Food & Drink Shop,Mediterranean Restaurant,Gastropub,Brazilian Restaurant,Sandwich Place,Chinese Restaurant
4,Cambridge,East Cambridge,42.369655,-71.088493,0,Café,American Restaurant,Fish Market,Coffee Shop,Italian Restaurant,Comfort Food Restaurant,Taco Place,Sushi Restaurant,Breakfast Spot,Brewery
5,Cambridge,Cambridge Highlands,42.395856,-71.156412,0,Gourmet Shop,Deli / Bodega,Trail,Donut Shop,Art Gallery,Pet Store,Arts & Crafts Store,Hardware Store,Athletics & Sports,Yoga Studio
6,Cambridge,West Cambridge,42.386807,-71.141027,0,Mattress Store,Bakery,Pet Store,Grocery Store,Gym,Mobile Phone Shop,Liquor Store,Yoga Studio,Sporting Goods Shop,Shopping Mall


# 7. Select the best neighborhood

In [40]:
#Print the top 5 venues in the clustered cambridge neighborhoods
top_5_cambridge=chosen_cluster_cambridge[chosen_cluster_cambridge.columns[0:10]]
top_5_cambridge

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Cambridge,Area 2/MIT,42.364903,-71.089729,0,Café,Bar,Coffee Shop,Italian Restaurant,Hotel
1,Cambridge,Mid-Cambridge,42.378088,-71.107275,0,Grocery Store,Coffee Shop,Climbing Gym,New American Restaurant,Bus Stop
2,Cambridge,Wellington-Harrington,42.371835,-71.098528,0,Coffee Shop,Park,Bar,New American Restaurant,Food & Drink Shop
3,Cambridge,Area Four,42.371835,-71.098528,0,Coffee Shop,Park,Bar,New American Restaurant,Food & Drink Shop
4,Cambridge,East Cambridge,42.369655,-71.088493,0,Café,American Restaurant,Fish Market,Coffee Shop,Italian Restaurant
5,Cambridge,Cambridge Highlands,42.395856,-71.156412,0,Gourmet Shop,Deli / Bodega,Trail,Donut Shop,Art Gallery
6,Cambridge,West Cambridge,42.386807,-71.141027,0,Mattress Store,Bakery,Pet Store,Grocery Store,Gym


In [41]:
#find the frequency of coffee shops in all neighborhoods
coffee=all_grouped[['Neighborhood','Coffee Shop']]
coffee.head()

,Neighborhood,Coffee Shop
0,Agassiz,0.058824
1,Area 2/MIT,0.051282
2,Area Four,0.076923
3,Battery Park City,0.070000
4,Cambridge Highlands,0.000000


In [42]:
#join the coffe shop frequency with the clustered cambridge information
coffee_freq=pd.merge(top_5_cambridge,
                    coffee,
                    left_on='Neighborhood',
                    right_on='Neighborhood',
                    how='inner')
coffee_freq.sort_values('Coffee Shop', ascending=False)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,Coffee Shop
1,Cambridge,Mid-Cambridge,42.378088,-71.107275,0,Grocery Store,Coffee Shop,Climbing Gym,New American Restaurant,Bus Stop,0.080000
2,Cambridge,Wellington-Harrington,42.371835,-71.098528,0,Coffee Shop,Park,Bar,New American Restaurant,Food & Drink Shop,0.076923
3,Cambridge,Area Four,42.371835,-71.098528,0,Coffee Shop,Park,Bar,New American Restaurant,Food & Drink Shop,0.076923
4,Cambridge,East Cambridge,42.369655,-71.088493,0,Café,American Restaurant,Fish Market,Coffee Shop,Italian Restaurant,0.055556
0,Cambridge,Area 2/MIT,42.364903,-71.089729,0,Café,Bar,Coffee Shop,Italian Restaurant,Hotel,0.051282
6,Cambridge,West Cambridge,42.386807,-71.141027,0,Mattress Store,Bakery,Pet Store,Grocery Store,Gym,0.029412
5,Cambridge,Cambridge Highlands,42.395856,-71.156412,0,Gourmet Shop,Deli / Bodega,Trail,Donut Shop,Art Gallery,0.000000


The cambridge neighborhood that is the most similar to Morningside Heights and has the highest frequency of coffee shops is at the top of the coffee_freq dataframe.